# Ryan Langlois
# CS 3110/5110: Data Privacy
## Final Project - Differentially Private Gradient Descent

In [136]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

df = pd.read_csv('https://raw.githubusercontent.com/UVMRyan/DataPrivacyFinal/refs/heads/main/student-mat.csv')

In [137]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [138]:
# Convert binary columns to ones and zeros
df['school'] = df['school'].apply(lambda x: 1 if x == 'GP' else 0)
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'M' else 0)
df['address'] = df['address'].apply(lambda x: 1 if x == 'U' else 0)
df['famsize'] = df['famsize'].apply(lambda x: 1 if x == 'GT3' else 0)
df['Pstatus'] = df['Pstatus'].apply(lambda x: 1 if x == 'T' else 0)
df['schoolsup'] = df['schoolsup'].apply(lambda x: 1 if x == 'yes' else 0)
df['famsup'] = df['famsup'].apply(lambda x: 1 if x == 'yes' else 0)
df['paid'] = df['paid'].apply(lambda x: 1 if x == 'yes' else 0)
df['activities'] = df['activities'].apply(lambda x: 1 if x == 'yes' else 0)
df['nursery'] = df['nursery'].apply(lambda x: 1 if x == 'yes' else 0)
df['higher'] = df['higher'].apply(lambda x: 1 if x == 'yes' else 0)
df['internet'] = df['internet'].apply(lambda x: 1 if x == 'yes' else 0)
df['romantic'] = df['romantic'].apply(lambda x: 1 if x == 'yes' else 0)

In [139]:
# One hot encode categorical columns
df = pd.get_dummies(df, dtype = int)

In [140]:
# Convert output column to category
df['G3'] = df['G3'].apply(lambda x: 1 if x >= 10 else 0)

In [141]:
input_cols = [c for c in df.columns if c != 'G3' and c != 'G2' and c != 'G1']
output_cols = ['G3']
X = df[input_cols].to_numpy()
y = df[output_cols].to_numpy()

In [142]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size].ravel()
y_test = y[training_size:].ravel()

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 316 79


In [121]:
X_test.shape

(79, 43)

## Using scikit-learn

In [143]:
from sklearn.linear_model import LogisticRegression

In [144]:
def train_model():
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model

model = train_model()
print('Model coefficients:', dict(zip(input_cols, model.coef_[0])))
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])

Model coefficients: {'school': 0.5238257793923073, 'sex': 0.39444693349858717, 'age': -0.09009349879090583, 'address': 0.3029938047129259, 'famsize': -0.002604638014613089, 'Pstatus': -0.28817160826614446, 'Medu': 0.042969815440269586, 'Fedu': -0.024970745420573943, 'traveltime': -0.037536900413689664, 'studytime': 0.07938215538692803, 'failures': -0.7169414074071807, 'schoolsup': -0.721487628075231, 'famsup': -0.481511548718458, 'paid': 0.04507109160672931, 'activities': 0.08248457212606908, 'nursery': 0.0866788726921105, 'higher': 0.9621615245983579, 'internet': 0.1958989948110455, 'romantic': -0.2179480674153192, 'famrel': 0.3311704883280583, 'freetime': 0.12883826858558575, 'goout': -0.6211705002583018, 'Dalc': -0.12110545729411545, 'Walc': 0.26357647155286595, 'health': -0.18503437457808444, 'absences': -0.02617542460359337, 'Mjob_at_home': -0.21012567667468585, 'Mjob_health': 0.43469533090671836, 'Mjob_other': 0.05977960672238152, 'Mjob_services': 0.5892416690145418, 'Mjob_teache

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Average gradient of the loss

In [145]:
# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

In [146]:
def avg_grad(theta, X, y):
    # list of vectors, each vector has length 104
    all_grads = [gradient(theta, X[i], y[i]) for i in range(len(X))]
    # compute the column-wise average
    avg_grad = np.mean(all_grads, axis=0)
    return avg_grad

## Gradient descent algorithm

In [147]:
def gradient_descent(iterations):
    theta = [0 for _ in range(43)] # initial model
    for _ in range(iterations):
        theta = theta - avg_grad(theta, X_train, y_train)
    return theta

theta = gradient_descent(50)
theta

array([0.34018987, 0.18196203, 5.53797468, 0.28797468, 0.23892405,
       0.30063291, 0.97310127, 0.87974684, 0.46518987, 0.68987342,
       0.05696203, 0.0443038 , 0.21044304, 0.16772152, 0.18829114,
       0.27689873, 0.33227848, 0.28955696, 0.09335443, 1.36075949,
       1.09018987, 0.99050633, 0.47310127, 0.74050633, 1.18829114,
       1.8306962 , 0.03797468, 0.03797468, 0.11867089, 0.09493671,
       0.05063291, 0.01265823, 0.01898734, 0.19778481, 0.0806962 ,
       0.03006329, 0.1028481 , 0.09493671, 0.03006329, 0.11234177,
       0.08227848, 0.23734177, 0.02056962])

In [148]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

accuracy(theta)

0.6329113924050633

## Noisy gradient descent algorithm

In [151]:
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)

    if norm > b:
        return b * (v / norm)
    else:
        return v

def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.array([0 for _ in range(43)]) # initial model
    epsilon_i = epsilon / iterations
    delta_i = delta / iterations
    for _ in range(iterations):

        # 1. calculate gradients
        all_grads = [gradient(theta, X_train[i], y_train[i]) for i in range(len(X_train))]

        # 2. call L2_clip on each gradient
        b = 3
        clipped_grads = [L2_clip(g, b) for g in all_grads]

        # 3. take the sum of the clipped gradients and add noise
        grad_sum = np.sum(clipped_grads, axis=0)

        # Sensitivity is correct, by clipping
        noisy_grad_sum = gaussian_mech_vec(grad_sum,
                                           sensitivity=b,
                                           epsilon=epsilon_i,
                                           delta=delta_i)
        # danger: reveals the size of the training data
        # probably this is not a big deal to reveal
        noisy_grad = np.array(noisy_grad_sum) / len(X_train)
        theta = theta - noisy_grad
    return theta

theta = noisy_gradient_descent(50, 1, 1e-5)
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.6329113924050633


## Second dataset

In [153]:
df = pd.read_csv('https://raw.githubusercontent.com/UVMRyan/DataPrivacyFinal/refs/heads/main/student-por.csv')

# Convert binary columns to ones and zeros
df['school'] = df['school'].apply(lambda x: 1 if x == 'GP' else 0)
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'M' else 0)
df['address'] = df['address'].apply(lambda x: 1 if x == 'U' else 0)
df['famsize'] = df['famsize'].apply(lambda x: 1 if x == 'GT3' else 0)
df['Pstatus'] = df['Pstatus'].apply(lambda x: 1 if x == 'T' else 0)
df['schoolsup'] = df['schoolsup'].apply(lambda x: 1 if x == 'yes' else 0)
df['famsup'] = df['famsup'].apply(lambda x: 1 if x == 'yes' else 0)
df['paid'] = df['paid'].apply(lambda x: 1 if x == 'yes' else 0)
df['activities'] = df['activities'].apply(lambda x: 1 if x == 'yes' else 0)
df['nursery'] = df['nursery'].apply(lambda x: 1 if x == 'yes' else 0)
df['higher'] = df['higher'].apply(lambda x: 1 if x == 'yes' else 0)
df['internet'] = df['internet'].apply(lambda x: 1 if x == 'yes' else 0)
df['romantic'] = df['romantic'].apply(lambda x: 1 if x == 'yes' else 0)

# One hot encode categorical columns
categorical_columns = ['Mjob', 'Fjob', 'reason', 'guardian']
df = pd.get_dummies(df, dtype = int)

# Convert output column to category
df['G3'] = df['G3'].apply(lambda x: 1 if x >= 10 else 0)

input_cols = [c for c in df.columns if c != 'G3' and c != 'G2' and c != 'G1']
output_cols = ['G3']
X = df[input_cols].to_numpy()
y = df[output_cols].to_numpy()

# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size].ravel()
y_test = y[training_size:].ravel()

print('Train and test set sizes:', len(y_train), len(y_test))

theta = noisy_gradient_descent(50, 1, 1e-5)
print('Final accuracy:', accuracy(theta))

Train and test set sizes: 519 130
Final accuracy: 0.6615384615384615
